Реализуйте задачу классификации на основе BERT-like модели и KNN на данных Russian Intents Dataset с Kaggle.

Код для создания поискового векторного индекса + логика определения класса на основе близости к обучающим объектам (по ближайшему, по топ-N ближайших, и т. п.).

Для этого:

1. скачайте и загрузите данные из набора Russian Intents Dataset и разделите их на части для обучения и тестирования;
2. Обработайте текст: удалите лишние шумы и символы из текста при необходимости, разбейте текст на отдельные слова;
3. возьмите готовую модель BERT и доработайте её под задачу классификации на данных Russian Intents Dataset;
4. создайте поисковый индекс;
5. определите класс по близости;
6. Проверьте точность классификации на ваших тестовых данных.

In [9]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

In [10]:
# Download latest version
path_data = kagglehub.dataset_download("constantinwerner/qa-intents-dataset-university-domain")

print("Path to dataset files:", path_data)

Path to dataset files: /kaggle/input/qa-intents-dataset-university-domain


In [11]:
# Прочитать данные и разделить на выборки
data = pd.read_csv(f'{path_data}/dataset_train.tsv',  sep='\t', header=None, names=['text', 'label'])

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data

,text,label
11365,расписание преподавателя Коваленко Л.В.,sched_teacher
1937,будкер расположен где,loc_nuclphysinstitute
5875,какие способы оплаты проживание в своем общежи...,pay_offline_dorm
1031,больница раскопать,loc_clinic
12607,подключить профсоюз студентов,student_trade_union_enter
...,...,...
11964,где сейчас Бузмакова О.Г.,sched_teacher
5191,что нужно для жизни в общежития скажи плиз,dorm_living
5390,закупаться продукт?,loc_shop
860,бухгалтерия где есть,loc_accounting


In [12]:
# Обработка данных
nltk.download('punkt')
nltk.download('stopwords')

nltk.download('punkt_tab')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('russian')]
    return " ".join(tokens)

train_data['cleaned_text'] = train_data['text'].apply(preprocess_text)
test_data['cleaned_text'] = test_data['text'].apply(preprocess_text)
train_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,text,label,cleaned_text
11365,расписание преподавателя Коваленко Л.В.,sched_teacher,расписание преподавателя коваленко лв
1937,будкер расположен где,loc_nuclphysinstitute,будкер расположен
5875,какие способы оплаты проживание в своем общежи...,pay_offline_dorm,какие способы оплаты проживание своем общежити...
1031,больница раскопать,loc_clinic,больница раскопать
12607,подключить профсоюз студентов,student_trade_union_enter,подключить профсоюз студентов
...,...,...,...
11964,где сейчас Бузмакова О.Г.,sched_teacher,бузмакова ог
5191,что нужно для жизни в общежития скажи плиз,dorm_living,нужно жизни общежития скажи плиз
5390,закупаться продукт?,loc_shop,закупаться продукт
860,бухгалтерия где есть,loc_accounting,бухгалтерия


In [13]:
# Получить эмбеддингы с помощью RuBERT
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

train_embeddings = get_bert_embeddings(train_data['cleaned_text'].tolist())
test_embeddings = get_bert_embeddings(test_data['cleaned_text'].tolist())

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Создать KNN-индекса
knn_model = NearestNeighbors(n_neighbors=3, metric='cosine')
knn_model.fit(train_embeddings)

NearestNeighbors(metric='cosine', n_neighbors=3)

In [15]:
# Определить класс по близости
def classify_with_knn(query_embedding, k=3):
    distances, indices = knn_model.kneighbors(query_embedding.reshape(1, -1), n_neighbors=k)
    nearest_labels = train_data.iloc[indices[0]]['label'].values
    predicted_label = max(set(nearest_labels), key=list(nearest_labels).count)
    return predicted_label

In [16]:
# Классификация тестовых данных
predicted_labels = [classify_with_knn(embedding) for embedding in test_embeddings]

# Оценка точности
accuracy = accuracy_score(test_data['label'], predicted_labels)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.802
